# Convertendo os valores

In [1]:
import pandas as pd
import re
import numpy as np

df = pd.read_excel('amostras_transformadas.xlsx')

def conversor(valor):
    numeros = re.search(r'[\d.,]+', valor).group().replace(',', '.')  # Extraindo somente os números
    if "µg/L" in valor:
        valor_convertido = float(numeros) / 1000  # Convertendo as unidades µg/L para mg/L
        valor_convertido_str = str(valor_convertido).replace('.', ',')  # Gerando o padrão numerico do código
        return valor_convertido_str + ' mg/L'  # Adicionando a unidade mg/L
    return valor  # retornando o valor original, se o valor não foi convertido de µg/L para mg/L

# convertendo todos os valores para strings
df["Valor"] = df["Valor"].astype(str)

# aplicando a função na coluna Valor
df["Valor"] = df["Valor"].apply(conversor)

# realizando o split baseado no espaço entre valor e unidade
df[['Valor', 'Unidades']] = df['Valor'].str.split(' ', expand=True)

In [2]:
df.to_excel('Dados_.xlsx', index=False)

# Criando as últimas colunas

In [3]:
refs = pd.read_excel('20230701_valores_referencia.xlsx')

#removendo as unidades
refs['codigo_parametro'] = refs['codigo_parametro'].str.split('\(').str[0].str.strip() 

#convertendo para o padrão numeríco do python
df["Valor"] = df["Valor"].str.replace("<", "").str.replace(",", ".").astype(float) 

# Mesclando os DataFrames com base no atributo
df_refs = pd.merge(df, refs, left_on="Atributo", right_on="codigo_parametro", how="left")

# Função para comparar os valores das análises e quantificação dos elementos químicos
def comparar_elementos(row):
    if pd.isnull(row["valor_maximo"]):
        return 'Sem referência'
    elif row["Atributo"] == 'pH': #uma comparação específica para o pH
        if 6 <= row["Valor"] <= 9:
            return "Conforme"
        else:
            return "Inconforme"
    elif row["Atributo"] == 'Saturação de OD': #uma comparação específica para a Saturação de Oxigênio Dissolvido
        if 0 <= row["Valor"] <= 4:
            return "Conforme"
        else:
            return "Inconforme"
    elif row["Valor"] < row["valor_maximo"]:
        return "Conforme" #considerando que são análises para fiscalização de água, segundo a legislação da ANVISA
    elif row["Valor"] > row["valor_maximo"]:
        return "Inconforme"
    else:
        return "Inconforme"
    
# Aplicando a função
df_refs["Resultado"] = df_refs.apply(comparar_elementos, axis=1)

#removendo os códigos do dataframe
df_refs.drop(['codigo_parametro'],axis=1, inplace=True)

#criando a coluna limite de quantificação
df_refs["Limite de quantificação"] = np.where(
    df_refs["valor_minimo"].notnull() & df_refs["valor_maximo"].notnull(),
    df_refs["valor_minimo"].astype(str) + " a " + df_refs["valor_maximo"].astype(str),
    np.where(
        df_refs["valor_minimo"].notnull(),
        df_refs["valor_minimo"].astype(str),
        df_refs["valor_maximo"].astype(str),
    ),
)

#Limpando dados não solicitados pelo exercicio
df_refs.drop(['valor_minimo','valor_maximo'],axis=1, inplace=True)

#convertendo para o padrão brasileiro de decimais
df_refs.replace('.',',', inplace=True)

#substituindo os nan por Não quantificado
df_refs.replace('nan','Não quantificado', inplace=True)

#salvando o dataframe como excel
df_refs.to_excel('Dados_Final.xlsx', index=False)

# Para o exercicio 2 extra

Houve a extração dos dados no link http://antigo.anvisa.gov.br/documents/10181/2718376/RDC_717_2022_.pdf/46974199-1976-43d8-8a0d-565152cbeada no excel. As tabelas utilizadas foram manualmente transformadas para o padrão de acordo com a planilha de referência.Houve então uma comparação entre os dados da anvisa e os valores de referência, com os dados do laudo. Alguns dados presentes na legislação da ANVISA e nos valores de referência foram substituidos pelos valores da legislação, outros novos foram adicionados, pois não estavam nos valores de referência.

In [4]:
df = pd.read_excel('amostras_transformadas.xlsx')

df["Valor"] = df["Valor"].astype(str)

# aplicando a função na coluna Valor
df["Valor"] = df["Valor"].apply(conversor)

# realizando o split baseado no espaço entre valor e unidade
df[['Valor', 'Unidades']] = df['Valor'].str.split(' ', expand=True)

refs = pd.read_excel("ANVISA_valores_referencia - Copia.xlsx")

refs['codigo_parametro'] = refs['codigo_parametro'].str.split('\(').str[0].str.strip() 

#convertendo para o padrão numeríco do python
df["Valor"] = df["Valor"].str.replace("<", "").str.replace(",", ".").astype(float) 


# Mesclando os DataFrames com base no atributo
df_refs = pd.merge(df, refs, left_on="Atributo", right_on="codigo_parametro", how="left")

df_refs["valor_maximo"] = pd.to_numeric(df_refs["valor_maximo"], errors="coerce")

# Convert "Valor" column to numeric type
df_refs["Valor"] = pd.to_numeric(df_refs["Valor"], errors="coerce")
    
# Aplicando a função
df_refs["Resultado"] = df_refs.apply(comparar_elementos, axis=1)

#removendo os códigos do dataframe
df_refs.drop(['codigo_parametro'],axis=1, inplace=True)

#criando a coluna limite de quantificação
df_refs["Limite de quantificação"] = np.where(
    df_refs["valor_minimo"].notnull() & df_refs["valor_maximo"].notnull(),
    df_refs["valor_minimo"].astype(str) + " a " + df_refs["valor_maximo"].astype(str),
    np.where(
        df_refs["valor_minimo"].notnull(),
        df_refs["valor_minimo"].astype(str),
        df_refs["valor_maximo"].astype(str),
    ),
)

#Limpando dados não solicitados pelo exercicio
df_refs.drop(['valor_minimo','valor_maximo'],axis=1, inplace=True)

#convertendo para o padrão brasileiro de decimais
df_refs.replace('.',',', inplace=True)

#substituindo os nan por Não quantificado
df_refs.replace('nan','Não quantificado', inplace=True)

#salvando o dataframe como excel
df_refs.to_excel('Dados_Final_anvisa.xlsx', index=False)